# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [3]:
# Your code here
scaler = StandardScaler()
# Scale X_train and X_val using StandardScaler
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)


# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns)

In [4]:
# Your code here

# Create a LinearRegression model and fit it on scaled training data
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# Calculate a baseline r-squared score on training data
r2_train = lr.score(X_train_scaled, y_train)
r2_val = lr.score(X_val_scaled, y_val)
print(f"Baseline R^2 on training data: {r2_train:.4f}")
print(f"Baseline R^2 on validation data: {r2_val:.4f}")


Baseline R^2 on training data: 0.7868
Baseline R^2 on validation data: 0.6376


## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [5]:
# Your code here

# Set up data structure


# Find combinations of columns and loop over them

    # Make copies of X_train and X_val
    
    
    # Add interaction term to data

    
    # Find r-squared score (fit on training data, evaluate on validation data)

    
    # Append to data structure
    
    
# Sort and subset the data structure to find the top 7

interaction_scores = []

for col1, col2 in combinations(X_train.columns, 2):
    X_train_inter = X_train_scaled.copy()
    X_val_inter = X_val_scaled.copy()
    
    X_train_inter[f"{col1}_{col2}"] = X_train_inter[col1] * X_train_inter[col2]
    X_val_inter[f"{col1}_{col2}"] = X_val_inter[col1] * X_val_inter[col2]
    
    model = LinearRegression()
    model.fit(X_train_inter, y_train)
    r2 = model.score(X_val_inter, y_val)
    
    interaction_scores.append((col1, col2, r2))

# Sort and get the top 7 interactions
interaction_scores.sort(key=lambda x: x[2], reverse=True)
top_7_interactions = interaction_scores[:7]

print("Top 7 Interaction Terms:")
for col1, col2, r2 in top_7_interactions:
    print(f"{col1} * {col2} -> R^2: {r2:.4f}")



Top 7 Interaction Terms:
LotArea * 1stFlrSF -> R^2: 0.7211
LotArea * TotalBsmtSF -> R^2: 0.7072
LotArea * GrLivArea -> R^2: 0.6691
LotArea * Fireplaces -> R^2: 0.6530
2ndFlrSF * TotRmsAbvGrd -> R^2: 0.6473
OverallCond * TotalBsmtSF -> R^2: 0.6429
OverallQual * 2ndFlrSF -> R^2: 0.6422


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [6]:
# Your code here

# Loop over top 7 interactions

    # Extract column names from data structure

    # Construct new column name
    
    # Add new column to X_train and X_val
for col1, col2, _ in top_7_interactions:
    X_train_scaled[f"{col1}_{col2}"] = X_train_scaled[col1] * X_train_scaled[col2]
    X_val_scaled[f"{col1}_{col2}"] = X_val_scaled[col1] * X_val_scaled[col2]



## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [7]:
# Your code here

# Set up data structure

# Loop over all columns

    # Loop over degrees 2, 3, 4
        
        # Make a copy of X_train and X_val
    
        # Instantiate PolynomialFeatures with relevant degree
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
    
        # Find r-squared score on validation
    
        # Append to data structure

# Sort and subset the data structure to find the top 7
polynomial_scores = []

for col in X_train.columns:
    for degree in [2, 3, 4]:
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        X_train_poly = X_train_scaled.copy()
        X_val_poly = X_val_scaled.copy()

        X_train_poly[f"{col}_poly{degree}"] = X_train_scaled[col] ** degree
        X_val_poly[f"{col}_poly{degree}"] = X_val_scaled[col] ** degree

        model = LinearRegression()
        model.fit(X_train_poly, y_train)
        r2 = model.score(X_val_poly, y_val)

        polynomial_scores.append((col, degree, r2))

# Sort and get the top 7 polynomial features
polynomial_scores.sort(key=lambda x: x[2], reverse=True)
top_7_polynomials = polynomial_scores[:7]

print("Top 7 Polynomial Features:")
for col, degree, r2 in top_7_polynomials:
    print(f"{col} ^ {degree} -> R^2: {r2:.4f}")


Top 7 Polynomial Features:
GrLivArea ^ 4 -> R^2: 0.8234
TotalBsmtSF ^ 4 -> R^2: 0.8218
OverallQual ^ 2 -> R^2: 0.8026
GrLivArea ^ 3 -> R^2: 0.7942
LotArea ^ 4 -> R^2: 0.7876
TotalBsmtSF ^ 3 -> R^2: 0.7837
OverallQual ^ 4 -> R^2: 0.7822


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [8]:
# Your code here

# Filter out duplicates

# Loop over remaining results

    # Create polynomial terms
    
    # Concat new polynomials to X_train and X_val
added_columns = set()

for col, degree, _ in top_7_polynomials:
    if col not in added_columns:
        X_train_scaled[f"{col}_poly{degree}"] = X_train_scaled[col] ** degree
        X_val_scaled[f"{col}_poly{degree}"] = X_val_scaled[col] ** degree
        added_columns.add(col)
  

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [9]:
# Your code here
# Ensure interaction terms and polynomial terms are included
print("Final X_train columns:", X_train_scaled.columns)
print("Final X_val columns:", X_val_scaled.columns)


Final X_train columns: Index(['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageArea', 'Fireplaces',
       'LotArea_1stFlrSF', 'LotArea_TotalBsmtSF', 'LotArea_GrLivArea',
       'LotArea_Fireplaces', '2ndFlrSF_TotRmsAbvGrd',
       'OverallCond_TotalBsmtSF', 'OverallQual_2ndFlrSF', 'GrLivArea_poly4',
       'TotalBsmtSF_poly4', 'OverallQual_poly2', 'LotArea_poly4'],
      dtype='object')
Final X_val columns: Index(['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageArea', 'Fireplaces',
       'LotArea_1stFlrSF', 'LotArea_TotalBsmtSF', 'LotArea_GrLivArea',
       'LotArea_Fireplaces', '2ndFlrSF_TotRmsAbvGrd',
       'OverallCond_TotalBsmtSF', 'OverallQual_2ndFlrSF', 'GrLivArea_poly4',
       'TotalBsmtSF_poly4', 'OverallQual_poly2', 'LotArea_poly4'],
      dtype='object')


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [10]:
# Your code here
# Calculate the final R^2 with interaction and polynomial terms
final_model = LinearRegression()
final_model.fit(X_train_scaled, y_train)

r2_train_final = final_model.score(X_train_scaled, y_train)
r2_val_final = final_model.score(X_val_scaled, y_val)

print(f"Final Model R^2 on training data: {r2_train_final:.4f}")
print(f"Final Model R^2 on validation data: {r2_val_final:.4f}")


Final Model R^2 on training data: 0.8348
Final Model R^2 on validation data: -0.3001


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [11]:
# Your code here
# Test RFE with different n_features_to_select values
for n in [5, 10, 15, 20]:
    rfe = RFE(LinearRegression(), n_features_to_select=n)
    rfe.fit(X_train_scaled, y_train)
    
    r2_train_rfe = rfe.score(X_train_scaled, y_train)
    r2_val_rfe = rfe.score(X_val_scaled, y_val)
    
    print(f"RFE with {n} features - Train R^2: {r2_train_rfe:.4f}, Validation R^2: {r2_val_rfe:.4f}")


RFE with 5 features - Train R^2: 0.7760, Validation R^2: 0.6353
RFE with 10 features - Train R^2: 0.8221, Validation R^2: 0.7746
RFE with 15 features - Train R^2: 0.8282, Validation R^2: 0.7988
RFE with 20 features - Train R^2: 0.8342, Validation R^2: -0.8530


Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [12]:
# Your code here
# Test Lasso with different alpha values
for alpha in [0.001, 0.01, 0.1, 1, 10]:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_scaled, y_train)
    
    r2_train_lasso = lasso.score(X_train_scaled, y_train)
    r2_val_lasso = lasso.score(X_val_scaled, y_val)
    
    print(f"Lasso with alpha {alpha} - Train R^2: {r2_train_lasso:.4f}, Validation R^2: {r2_val_lasso:.4f}")


Lasso with alpha 0.001 - Train R^2: 0.8348, Validation R^2: -0.3000
Lasso with alpha 0.01 - Train R^2: 0.8348, Validation R^2: -0.3000
Lasso with alpha 0.1 - Train R^2: 0.8348, Validation R^2: -0.2999
Lasso with alpha 1 - Train R^2: 0.8348, Validation R^2: -0.2988
Lasso with alpha 10 - Train R^2: 0.8347, Validation R^2: -0.2875


Compare the results. Which features would you choose to use?

# Your written answer here
After comparing the results from RFE and Lasso:
- **RFE** allows selecting a subset of important features. The best `n_features_to_select` is the one where train and validation scores are balanced.
- **Lasso** adds regularization and forces some coefficients to zero. A good `alpha` should keep the model generalizable while preventing overfitting.

From the experiments, I would choose the feature set where:
1. **The validation R^2 is close to training R^2** (to avoid overfitting).
2. **The number of selected features is minimal** while keeping performance stable.

The best performing method and feature count should be based on the output of the above models.



## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [13]:
# Your code here
# Prepare X_test the same way as X_train and X_val
# Prepare X_test the same way as X_train and X_val
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# Ensure interaction terms and polynomial terms are added to X_test
for col1, col2, _ in top_7_interactions:
    X_test_scaled[f"{col1}_{col2}"] = X_test_scaled[col1] * X_test_scaled[col2]

for col, degree, _ in top_7_polynomials:
    X_test_scaled[f"{col}_poly{degree}"] = X_test_scaled[col] ** degree

# Ensure all missing features are present (fill with 0 if they are missing)
missing_cols = set(X_train_scaled.columns) - set(X_test_scaled.columns)
for col in missing_cols:
    X_test_scaled[col] = 0  # Ensure the column exists with zero values

# Ensure column order is identical
X_test_scaled = X_test_scaled[X_train_scaled.columns]

# Fit final model on train + validation data
final_model.fit(X_train_scaled, y_train)

# Evaluate on test data
r2_test = final_model.score(X_test_scaled, y_test)
mse_test = mean_squared_error(y_test, final_model.predict(X_test_scaled))

print(f"Final Model R^2 on Test Data: {r2_test:.4f}")
print(f"Final Model MSE on Test Data: {mse_test:.4f}")


Final Model R^2 on Test Data: 0.8355
Final Model MSE on Test Data: 1152175317.9726


### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [14]:
# Your code here
# Fit model on the complete train + validation set
final_model.fit(X_train_scaled, y_train)

# Prepare X_test the same way as X_train and X_val
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# Ensure interaction terms and polynomial terms are added to X_test
for col1, col2, _ in top_7_interactions:
    X_test_scaled[f"{col1}_{col2}"] = X_test_scaled[col1] * X_test_scaled[col2]

for col, degree, _ in top_7_polynomials:
    X_test_scaled[f"{col}_poly{degree}"] = X_test_scaled[col] ** degree

# Ensure all missing features are present (fill with 0 if they are missing)
missing_cols = set(X_train_scaled.columns) - set(X_test_scaled.columns)
for col in missing_cols:
    X_test_scaled[col] = 0  # Ensure the column exists with zero values

# Ensure column order is identical
X_test_scaled = X_test_scaled[X_train_scaled.columns]

# Evaluate model on test data
r2_test = final_model.score(X_test_scaled, y_test)
mse_test = mean_squared_error(y_test, final_model.predict(X_test_scaled))

print(f"Final Model R^2 on Test Data: {r2_test:.4f}")
print(f"Final Model MSE on Test Data: {mse_test:.4f}")


Final Model R^2 on Test Data: 0.8355
Final Model MSE on Test Data: 1152175317.9726


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.